This code generates the model SNe for each model type. 

Check up on Model 4, I think the mixed map has a normalization error. (map built in SNeCO/3.4cModelVisualization)

In [1]:
#import packages and initialize settings
import time
import os
import numpy as np
from astropy.io import fits, ascii
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, join
from reproject import reproject_interp
import aplpy
import scipy
from scipy.interpolate import interp1d

plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

import sys
sys.path.append('./Python')
from nearest import findNearest
from deprojectGalaxy import deproject
from SNeGenerationModels import runModels
from getMapValue import getValue

In [2]:
#Get Galaxy names, distances, and fits file locations

# Unpack master table to get SNe information
GalaxyData = Table.read('../Data/0.MUSEData.csv', format='csv')
SampleData = Table.read('../Data/2.MUSESampleCat.csv', format='csv') 
SampleData.remove_column("Distance")
SampleData.remove_column("MUSENatMap")
SampleData.remove_column("MUSE150pcMap")
SampleData.remove_column("MUSEHII")
SampleData.remove_column("MUSERes")
SampleData.remove_column("IRACfiles")

database = join(GalaxyData, SampleData, keys = "Galaxy", join_type = 'inner')
centerCoords = [(database["GalRa"][i], database["GalDec"][i]) for i in range(len(database))]   

database[0:8]


Galaxy,Distance,GalRa,GalDec,PosAng,Incl,MUSENatRes,MUSEmapNat,MUSEmap150pc,MUSE_HII_reg,IRAC_files,Supernova,Type,Ra,Dec,InSample
str7,float64,float64,float64,float64,float64,float64,str71,str70,str50,str40,str11,str12,float64,float64,str4
NGC1087,15.85,41.60492,-0.498717,359.1,42.9,0.92,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1087-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,../Data/IRAC/ngc1087_irac1_atGauss4.fits,SN1995V,II,41.61152777777777,-0.4987861111111111,True
NGC1300,18.99,49.920815,-19.411114,278.0,31.8,0.89,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1300-0.89asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1300-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1300_nebulae_mask_V2.fits,../Data/IRAC/ngc1300_irac1_atGauss4.fits,SN2022acko,II,49.91245833333333,-19.39518888888889,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,../Data/IRAC/ngc1365_irac1_atGauss4.fits,SN2012fr,Ia,53.40057916666666,-36.12676944444445,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,../Data/IRAC/ngc1365_irac1_atGauss4.fits,SN2001du,II,53.3713125,-36.142111111111106,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,../Data/IRAC/ngc1365_irac1_atGauss4.fits,SN1983V,Ic,53.381876388888884,-36.14859166666667,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,../Data/IRAC/ngc1365_irac1_atGauss4.fits,SN1957C,Unclassified,53.38349999999999,-36.11770833333333,True
NGC1433,18.63,55.506195,-47.221943,199.7,28.6,0.91,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1433-0.91asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1433-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1433_nebulae_mask_V2.fits,../Data/IRAC/ngc1433_irac1_atGauss4.fits,SN1985P,II,55.52640625,-47.209987222222225,True
NGC1566,17.69,65.00159,-54.93801,214.7,29.5,0.8,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1566-0.80asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1566-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1566_nebulae_mask_V2.fits,../Data/IRAC/ngc1566_irac1_atGauss4.fits,SN2021aefx,Ia,64.9725,-54.94808055555555,True


In [3]:
# Functions
def angDistToPc(x,galDist):
    return(galDist*10**6*np.tan(x*np.pi/180))

def nearestHII(galaxy, HII_file, centerCoord, pa, incl, galDist, SNra, SNdec):
    HII_val = []
    if os.path.isfile(HII_file):

        # read in fits files
        hdu_HII = pyfits.open(HII_file)
        HIIMap  = hdu_HII[0].data

        #Convert x & y pixels to ra and dec
        wcs      = WCS(hdu_HII[0].header, naxis=2)
        naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
        grid     = np.indices((naxis[1],naxis[0]))
        ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)

        #deproject ra and dec to dx and dy
        radius, projang, dx, dy = deproject(center_coord=centerCoord, incl=incl, pa=pa, ra=ra, dec=dec, return_offset=True)

        # Get the value from the HII map at the SN location
        pix_x, pix_y = wcs.wcs_world2pix(SNra, SNdec, 0, ra_dec_order=True)
        pix_x = [int(x) for x in pix_x]
        pix_y = [int(x) for x in pix_y]
        
        for i in range(len(SNra)):
            HII_val.append(HIIMap[pix_y[i], pix_x[i]])

        
        #flatten data structures 
        f_HII  = HIIMap.flatten()
        f_ra   = ra.flatten()
        f_dec  = dec.flatten()    
        f_dx   = dx.flatten()
        f_dy   = dy.flatten()

        #remove nans
#         keep  = np.where(np.isfinite(f_HII))
        keep  = np.where(f_HII >= 0)
        map_ra    = f_ra[keep]
        map_dec   = f_dec[keep]
        map_HII   = f_HII[keep]
        map_dx    = f_dx[keep]
        map_dy    = f_dy[keep]

        SN_Rad, SN_PA, SN_dx, SN_dy = deproject(center_coord=centerCoord, incl=incl, pa=pa, ra=SNra, dec=SNdec, return_offset=True) 
        
        nearestHII_ang, idx = findNearest(map_dx, SN_dx, map_dy, SN_dy)
        HIIra, HIIdec = map_ra[idx], map_dec[idx]
        nearestHII_pc = angDistToPc(nearestHII_ang,galDist)
        
        #Check if the SN is in an HII region and adjust nearest HII region value accordingly
        for i in range(len(SNra)):
            if HII_val[i] > 0:
                nearestHII_pc[i] = 0.
        
#         print("Nearest HII region [pc]", nearestHII_pc, galaxy)

    else:
        print("No file for ", galaxy)

        nearestHII_pc = float("nan")
        HIIra = float("nan")
        HIIdec = float("nan")
        
    return(nearestHII_pc, HIIra, HIIdec)


In [4]:
#group sample by galaxy
gal_groups = database.group_by('Galaxy')
galaxies = gal_groups.groups.keys
indices = gal_groups.groups.indices
indices = indices.tolist()
indices.append(0)

gs, imgs, HIIimgs,IRACimgs, names, types, ras, decs, dists, res = [],[],[],[],[],[],[],[],[],[]
galRa,galDec,galPA,galIncl = [],[],[],[]

for i in range(len(indices)-2):
    SNnames, SNtypes, SNras, SNdecs = [],[],[],[]
    gs.append(gal_groups["Galaxy"][indices[i]])
    imgs.append(gal_groups["MUSEmapNat"][indices[i]])
    HIIimgs.append(gal_groups["MUSE_HII_reg"][indices[i]])
    IRACimgs.append(gal_groups["IRAC_files"][indices[i]])
    dists.append(gal_groups["Distance"][indices[i]])
    res.append(gal_groups["MUSENatRes"][indices[i]])
    galRa.append(gal_groups["GalRa"][indices[i]])
    galDec.append(gal_groups["GalDec"][indices[i]])
    galPA.append(gal_groups["PosAng"][indices[i]])
    galIncl.append(gal_groups["Incl"][indices[i]])
    
    #if gal_groups["SN_name"][indices[i]] != "SN1989B" and gal_groups["SN_name"][indices[i]] != "SN1973R":
    # Attach first SN per galaxy to a list
    SNnames.append(gal_groups["Supernova"][indices[i]])
    SNtypes.append(gal_groups["Type"][indices[i]])
    SNras.append(gal_groups["Ra"][indices[i]])
    SNdecs.append(gal_groups["Dec"][indices[i]])
    j = 1
    # attach additional  SN to list when galaxy has more than one
    while(indices[i] + j < indices[i+1]):    
        #if gal_groups["SN_name"][indices[i]+j] != "SN1989B" and gal_groups["SN_name"][indices[i]+j] != "SN1973R":
        SNnames.append(gal_groups["Supernova"][indices[i]+j])
        SNtypes.append(gal_groups["Type"][indices[i]+j])
        SNras.append(gal_groups["Ra"][indices[i]+j])
        SNdecs.append(gal_groups["Dec"][indices[i]+j])
        j += 1
    names.append(SNnames)
    types.append(SNtypes)
    ras.append(SNras)
    decs.append(SNdecs)

print(gs)
print(ras)

['NGC1087', 'NGC1300', 'NGC1365', 'NGC1433', 'NGC1566', 'NGC1672', 'NGC3627', 'NGC4254', 'NGC4303', 'NGC4321']
[[41.61152777777777], [49.91245833333333], [53.40057916666666, 53.3713125, 53.381876388888884, 53.38349999999999], [55.52640625], [64.9725, 64.99512499999999, 65.0058875], [71.423975, 71.45607916666667], [170.04809375, 170.05796666666666, 170.07957933333336, 170.07072916666667, 170.0592513888889], [184.71836111111114, 184.71066666666667, 184.71690625, 184.70291666666668], [185.48760972222217, 185.47536458333335, 185.50184583333336, 185.4698125, 185.46016458333332, 185.48041875, 185.48991666666666], [185.72488194444443, 185.74543194444445, 185.6970833333333, 185.72892708333336, 185.73393055555553]]


In [10]:
    
# # run models 1-4 and store results
galCol, counter, int_mod1, int_mod2, int_mod3, near_mod1, near_mod2, near_mod3 = [],[],[],[],[],[],[],[]
HaTot, HaHII, SDTot, SDHII = [],[],[],[]

for i in range(len(gs)):
# for i in range(0,1):
#     i = 8
    
    # Using MUSE map for coverage and to measure H-alpha emission
    img_hdu = pyfits.open(imgs[i])
    img_map = img_hdu["HA6562_FLUX"].data
    
    #Get IRAC maps to look at the stellar populations emission
    IRAC_hdu = pyfits.open(IRACimgs[i])
    IRAC_map = IRAC_hdu[0].data
    
    #Get HII regions masks
    hdu_HII = pyfits.open(HIIimgs[i])
    HIIMap  = hdu_HII[0].data    
    
    # Reproject IRAC maps
    IRAC_repro, IRAC_foot = reproject_interp(IRAC_hdu, img_hdu["HA6562_FLUX"].header)

    #Convert x & y pixels to ra and dec
    wcs      = WCS(hdu_HII[0].header, naxis=2)
    naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid     = np.indices((naxis[1],naxis[0]))
    ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)
            
    #flatten data structures 
    f_HII  = HIIMap.flatten()
    f_ra   = ra.flatten()
    f_dec  = dec.flatten()    
    f_img = img_map.flatten()
    f_IRAC = IRAC_repro.flatten() 
    
    #remove nans
    keep = np.where(np.isfinite(f_img) & np.isfinite(f_IRAC))
    map_HII  = f_HII[keep]
    map_IRAC = f_IRAC[keep]
    map_img  = f_img[keep]  
    
    # sum up the total emission
    HaTot.append(np.sum(map_img))
    SDTot.append(np.sum(map_IRAC))
    
    # Isolate HII regions
    keep  = np.where(map_HII >= 0)
    map_HII  = map_HII[keep]
    map_IRAC = map_IRAC[keep]
    map_img  = map_img[keep]
    
    # sum up the HII region emission
    HaHII.append(np.sum(map_img))
    SDHII.append(np.sum(map_IRAC))   
    
    # get maps with real values for models
#     keep = np.where(np.isfinite(f_img))
    keep = np.where(np.isfinite(f_img) & np.isfinite(f_IRAC))
    IRAC      = f_IRAC[keep]
    star_ra   = f_ra[keep]
    star_dec  = f_dec[keep]
    
    
    raArr1, decArr1, dxArr1, dyArr1, intArr1 = runModels(gs[i], imgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], modelType=1)
    raArr2, decArr2, dxArr2, dyArr2, intArr2 = runModels(gs[i], imgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], modelType=2)
    raArr3, decArr3, dxArr3, dyArr3, intArr3 = runModels(gs[i], imgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], modelType=3, starLight = IRAC, starRa = star_ra, starDec = star_dec)
    #raArr4, decArr4, dxArr4, dyArr4 = runModels(gals[i], mixedFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)

    n_mod1_dist, n_mod1_ra, n_mod1_dec = nearestHII(gs[i], HIIimgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], raArr1, decArr1)
    n_mod2_dist, n_mod2_ra, n_mod2_dec = nearestHII(gs[i], HIIimgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], raArr2, decArr2)
    n_mod3_dist, n_mod3_ra, n_mod3_dec = nearestHII(gs[i], HIIimgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], raArr3, decArr3)
    #n55_mod4, n65_mod4 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr4, decArr4)

    
    if (type(n_mod1_dist) != float and type(n_mod1_dist) != np.float64):
        for j in range(len(n_mod1_dist)):
            galCol.append(gs[i])
            counter.append(j)
            near_mod1.append(n_mod1_dist[j])
            near_mod2.append(n_mod2_dist[j])
            near_mod3.append(n_mod3_dist[j])
            #n55mod4.append(n55_mod4[j])
    else:
        for j in range(100):
            galCol.append(gals[i])
            counter.append(j)
            near_mod1.append(np.float('nan'))
            near_mod2.append(np.float('nan'))
            near_mod3.append(np.float('nan'))
            #n55mod4.append(np.float('nan'))  
            
    intArr1    = np.nan_to_num(np.clip(intArr1,0,np.max(intArr1)))
    intArr2    = np.nan_to_num(np.clip(intArr2,0,np.max(intArr2)))
    intArr3    = np.nan_to_num(np.clip(intArr3,0,np.max(intArr3)))
    
    int_mod1.append(intArr1)
    int_mod2.append(intArr2)
    int_mod3.append(intArr3)
    
#     #plotting code here
#     fig = plt.figure(figsize=(8, 8), facecolor = 'white')

#     f1 = aplpy.FITSFigure(img_map, figure=fig, subplot=(1,3,1))
#     f1.tick_labels.hide()
#     f1.ticks.hide()
#     f1.axis_labels.hide()
#     f1.set_title("Random", fontsize = 16)

#     f2 = aplpy.FITSFigure(img_map, figure=fig,  subplot=(1,3,2))
#     f2.tick_labels.hide()
#     f2.ticks.hide()
#     f2.axis_labels.hide()
#     f2.set_title("Follow H-alpha Distribution", fontsize=16)


#     f3 = aplpy.FITSFigure(IRAC_repro, stretch="log", figure=fig,  subplot=(1,3,3))
#     f3.tick_labels.hide()
#     f3.ticks.hide()
#     f3.axis_labels.hide()
# #     f3.show_contour(massMap, ifr_hdu, levels = [10**5.5, 10**6.5], colors = ["grey", "black"])
#     f3.set_title("Follow Stellar Disk", fontsize=16)

    
# #     f4 = aplpy.FITSFigure(mixMap, mixhdu, figure=fig,  subplot=(2,2,4))
# #     f4.tick_labels.hide()
# #     f4.ticks.hide()
# #     f4.axis_labels.hide()
# # #     f4.show_contour(massMap, ifr_hdu, levels = [10**5.5, 10**6.5], colors = ["grey", "black"])
# #     f4.set_title("Hybrid", fontsize=16)

    
#     f1.show_colorscale(vmin=0, pmax=95, cmap='magma', stretch='linear')
#     f1.set_nan_color('black')

#     f2.show_colorscale(vmin=0, pmax=95, cmap='magma', stretch='linear')
#     f2.set_nan_color('black')

#     f3.show_colorscale(vmin=0, pmax=95, cmap='magma', stretch='linear')
#     f3.set_nan_color('black')

# #     f4.show_colorscale(vmin=0, pmax=95, cmap='magma', stretch='linear')
# #     f4.set_nan_color('black')
    
#     fig.tight_layout()

#     xArr1, yArr1 = wcs.wcs_world2pix(raArr1,decArr1,0)
#     xArr2, yArr2 = wcs.wcs_world2pix(raArr2,decArr2,0)
#     xArr3, yArr3 = wcs.wcs_world2pix(raArr3,decArr3,0)
# #     xArr4, yArr4 = wcs.wcs_world2pix(raArr4,decArr4,0)

#     for k in range(len(dxArr3)):
#         f1.add_label(float(xArr1[k]), float(yArr1[k]), text='*', color='black', size = 22, weight="bold")
#         f2.add_label(float(xArr2[k]), float(yArr2[k]), text='*', color="black", weight="bold", size=22)
#         f3.add_label(float(xArr3[k]), float(yArr3[k]), text='*', color="black", weight="bold", size=22)
# #         f4.add_label(float(xArr4[k]), float(yArr4[k]), text='*', color="black", fontsize = 22, weight="bold")
#         f1.add_label(float(xArr1[k]), float(yArr1[k]), text='*', color='lime', size = 20)
#         f2.add_label(float(xArr2[k]), float(yArr2[k]), text='*', color="lime", size=20)
#         f3.add_label(float(xArr3[k]), float(yArr3[k]), text='*', color="lime", size=20)
# #         f4.add_label(float(xArr4[k]), float(yArr4[k]), text='*', color="lime", fontsize = 20)

#     figName = "../Figures/" + gs[i] + "Models.png"
#     fig.savefig(figName, dpi = 300)

    
int_mod1 = np.hstack(int_mod1)
int_mod2 = np.hstack(int_mod2)
int_mod3 = np.hstack(int_mod3)
    

# cdfSM_mod1 = np.sort(smMod1)/np.sum(smMod1)
# cdfSM_mod2 = np.sort(smMod2)/np.sum(smMod2) 
# cdfSM_mod3 = np.sort(smMod3)/np.sum(smMod3)
    
results = Table({"Galaxy": galCol,
                "ModelNum": counter,
                "near_Mod1_dist": near_mod1,
                "near_Mod2_dist": near_mod2,
                "near_Mod3_dist": near_mod3,
                "intMod1":int_mod1,
                "intMod2":int_mod2,
                "intMod3":int_mod3})

results.write("../Data/AllModels.csv", format = "csv", overwrite = True)

        
        
    

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
r_int: [3.49180816e+

r_int: [8.08233817e+01 2.71317618e+01 1.15865335e+02 5.03332989e+01
 6.07172570e+01 2.51644891e+02 4.33007742e+01 1.15648081e+02
 9.12260180e+01 1.07462222e+04 1.29118943e+01 2.09533147e+02
 3.56731214e+01 4.64792662e+03 4.03210609e+01 1.58386625e+02
 4.95958677e+01 6.06299415e+03 1.75394354e+01 2.51717727e+01
 1.09241981e+02 4.97157707e+01 4.39348837e+01 4.83636013e+01
 5.94845614e+02 4.54280073e+01 9.76594705e+00 5.34370819e+01
 1.70314628e+02 3.75873231e+01 1.31416925e+02 2.78835227e+02
 1.45804373e+04 1.50972663e+03 1.70069172e+02 1.93842795e+02
 4.61487570e+01 1.15309525e+03 2.08742583e+01 3.52519100e+01
 4.50284387e+02 5.19497488e+01 4.30436658e+01 1.70195671e+02
 1.69746589e+02 3.12089341e+01 5.14029291e+01 1.22299900e+01
 2.04948991e+02 1.74741365e+01 4.24430004e+01 1.45470092e+01
 4.16726696e+01 5.73859647e+01 7.00340700e+01 6.93168397e+01
 1.84963845e+02 7.52048957e+01 3.56619717e+01 5.55554217e+01
 2.78978950e+01 4.50279826e+01 3.07894863e+01 3.14498286e+01
 5.15013501e+01 1

r_int: [8.09913569e+01 6.87654454e+03 5.38228353e+02 4.57312768e+02
 1.82575537e+04 3.43049677e+01 1.20923550e+03 3.24795852e+02
 4.40261052e+03 2.00563238e+03 7.30110411e+03 3.65511529e+01
 5.91414230e+03 1.99903099e+03 2.59692997e+02 1.65023620e+03
 1.17089960e+03 4.53203713e+00 1.37158092e+03 1.89750586e+04
 5.25961669e+02 5.19463626e+03 5.93141340e+03 2.73964747e+04
 2.92291365e+03 2.49449353e+02 1.86230842e+01 3.61002680e+01
 3.01530865e+01 2.41988566e+02 3.79703629e+02 2.60604764e+03
 5.20249291e+02 4.20857482e+02 1.94508687e+03 5.17976015e+02
 2.96404899e+03 1.59728095e+03 1.79871760e+02 3.93872897e+01
 1.34779308e+03 6.18287821e+02 1.43060356e+03 1.76256340e+02
 6.76700400e+01 6.05405737e+02 7.31620998e+03 8.24024063e+01
 9.40709072e+02 1.01476456e+04 3.57101834e+01 6.23208897e+03
 4.23752810e+01 3.13610764e+02 3.44536446e+01 1.56685942e+02
 4.96859929e+02 1.02220815e+03 3.09362988e+01 1.98580667e+03
 1.51308998e+03 7.11448709e+01 1.86325171e+02 2.07854178e+03
 4.69362211e+02 5

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
r_int: [5.06763370e+

r_int: [2.30342010e+03 7.01679111e+03 2.98982512e+02 1.49952625e+03
 1.35819578e+02 8.82902037e+02 3.12997403e+02 6.81201154e+03
 8.86529398e+03 8.64736680e+02 9.22558167e+02 2.32660378e+02
 2.28544413e+03 2.59771078e+02 2.56764761e+03 1.14564273e+03
 2.89608416e+02 4.85978811e+03 1.23845124e+01 1.06830110e+02
 3.73959513e+02 1.34622260e+03 3.03404947e+02 2.63106075e+02
 2.80029780e+01 6.14932256e+02 2.99994525e+02 1.45696923e+03
 3.40758043e+02 2.59758320e+02 3.47760386e+03 1.70180307e+02
 5.06353948e+03 3.86254392e+02 3.28265592e+02 1.56130821e+03
 3.20002542e+03 2.19263259e+02 2.61512860e+02 1.48647736e+03
 3.38328449e+02 3.03182687e+02 4.42372148e+02 1.10669745e+02
 1.82399529e+02 2.00141611e+02 5.81263325e+02 1.37535779e+03
 2.17224963e+02 5.49493193e+02 6.80559641e+02 2.10673664e+02
 3.10293356e+02 2.43600512e+03 3.63457349e+03 1.81065739e+02
 4.00712865e+02 1.65965262e+03 8.46281771e+02 2.72298009e+04
 2.94607938e+03 7.97416906e+02 2.56267784e+02 4.23559051e+01
 5.81373515e+02 5

r_int: [9.44301041e+02 1.77068882e+03 7.97178312e+03 1.70872758e+04
 8.60848744e+03 4.92935905e+04 1.09062228e+04 1.42824880e+04
 1.16536391e+05 1.77742728e+03 1.32273508e+04 9.26664755e+04
 1.76275828e+05 3.73025753e+03 1.06051986e+03 8.86006191e+03
 9.41945365e+02 3.30666565e+03 1.09717771e+03 2.99427839e+03
 1.01081616e+04 1.77657032e+04 9.91995976e+03 1.95536064e+04
 6.29664989e+03 1.00659866e+04 2.94433317e+03 1.42062295e+05
 1.25189556e+04 2.81067312e+04 1.17404683e+05 1.18824232e+04
 1.15608829e+03 1.21126229e+04 2.89383379e+03 2.72934293e+04
 6.89457360e+02 1.53137435e+03 1.29142464e+04 3.12257127e+03
 5.31175667e+02 4.38569842e+02 1.82037762e+04 3.29608556e+04
 2.86725387e+04 1.98171833e+03 4.42563385e+04 2.13102449e+03
 4.31741731e+03 5.29251324e+03 1.84011183e+03 4.29353275e+03
 1.75940318e+03 3.71606834e+03 1.42754870e+03 2.29263029e+03
 1.85824439e+03 1.75084700e+03 9.73304333e+03 4.03958106e+04
 1.06709688e+03 6.88218195e+04 1.35314971e+04 6.99777594e+03
 2.71920992e+04 3

In [11]:
print("Total Ha emission:", np.sum(HaTot))
print("Halpha emisision in HII regions:", np.sum(HaHII))
print("Total SD emission:", np.sum(SDTot))
print("SD emisision in HII regions:", np.sum(SDHII))

print("Fraction of Ha Emission in HII regions",np.sum(HaHII)/np.sum(HaTot))
print("Fraction of SD Emission in HII regions",np.sum(SDHII)/np.sum(SDTot))

print(HaTot)

Total Ha emission: 7604300069.268526
Halpha emisision in HII regions: 4851714765.4856825
Total SD emission: 6980001.527765438
SD emisision in HII regions: 2238097.708438627
Fraction of Ha Emission in HII regions 0.6380225295281357
Fraction of SD Emission in HII regions 0.3206443006546342
[430542935.0201032, 179345534.81836388, 831085416.9293946, 185840759.71819738, 769252855.2159717, 1197806512.610021, 1019128630.2055503, 1205088503.7456038, 1133290499.7306232, 652918421.2746967]


In [ ]:
import math
# get the real SN sample's distances (copied over from 1c)
realSNe = [0,130,524,88,0,201,0,120,100,0,0,5,0,0,29,0,0,11,0,7,0,0,44,0,36,22,14,214,63,8,13,0,0]

# Here make CDFs of the distances from the generated SNe to the nearest HII regions
modelResults = Table.read("../Data/AllModels.csv", format = 'csv')

nm1 = [x for x in modelResults["near_Mod1_dist"] if math.isnan(x) == False]
nm2 = [x for x in modelResults["near_Mod2_dist"] if math.isnan(x) == False]
nm3 = [x for x in modelResults["near_Mod3_dist"] if math.isnan(x) == False]

# nm4 = np.concatenate((nm2[0:1914],nm3[1914:]))


In [ ]:
def findNear(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def makeCDFs(array):
    count, bins_count = np.histogram(array, bins=100)
    xVals = bins_count[1:]
    pdf = count / sum(count)
    cdf = np.cumsum(pdf)
    return(xVals, cdf)

def ecdf(data):
    """ Compute ECDF """
    x = np.sort(data)
    n = x.size
    y = np.arange(1, n+1) / n
    return(x,y)


In [ ]:
#original idea for how this should work
def randomPull(modelArr, numSNe=33, numRuns=1000):
    # pull intensity vals at locations of 33 model SNe 1000 times.    
    allPulls = []
    # iterate over 1000 runs
    for i in range(numRuns):
        singlePulls = []
        # iterate over number of SNe
        for j in range(numSNe):
            # choose a random distance from model SNe
            val = np.random.choice(modelArr)
            singlePulls.append(val)
        # sort the model SNe distances and interpolate then save
        sorted_pulls = np.sort(singlePulls)
        sortedPulls_y = np.sort(singlePulls)
        sortedPulls_x = np.linspace(0,len(sortedPulls_y),len(sortedPulls_y))
        y_f = interp1d(sortedPulls_x,sortedPulls_y,'linear')
        x = np.linspace(0,len(sortedPulls_y),1000)
        y = y_f(x)
        allPulls.append(y)        
        
    # Find the 1000 closest random pull values to each dist value in model SN array       
    allPer16Vals, allPer50Vals, allPer84Vals = [],[],[]
    for i in range(len(modelArr)):
        SNlocVals=[]
        for j in range(numRuns):
#             find the actual distance value that is the closest to the generated model values 
            SNlocVals.append(findNear(allPulls[j], modelArr[i]))
          
    # now find 16-84th percentiles from each of these local distributions around x value    
        locVals = np.sort(SNlocVals)
        per16Vals = np.percentile(locVals,16)
        allPer16Vals.append(per16Vals)
        per50Vals = np.percentile(locVals,50)     
        allPer50Vals.append(per50Vals)
        per84Vals = np.percentile(locVals,84)
        allPer84Vals.append(per84Vals)

    return(allPer16Vals, allPer50Vals, allPer84Vals)

In [ ]:
nSMm1_16, nSMm1_50, nSMm1_84 = randomPull(nm1)
nSMm2_16, nSMm2_50, nSMm2_84 = randomPull(nm2)
nSMm3_16, nSMm3_50, nSMm3_84 = randomPull(nm3)
# nSMm4_16, nSMm4_50, nSMm4_84 = randomPull(nm4)


# want to recreate ecdf plot with pyplot    
SNe,  ySNe  = ecdf(realSNe)
mod1, ymod1 = makeCDFs(nm1)
mod2, ymod2 = makeCDFs(nm2)
mod3, ymod3 = makeCDFs(nm3)
# mod4, ymod4 = makeCDFs(nSMm4)

mod1_16, ymod1_16 = makeCDFs(nSMm1_16)
mod1_50, ymod1_50 = makeCDFs(nSMm1_50)
mod1_84, ymod1_84 = makeCDFs(nSMm1_84)
mod2_16, ymod2_16 = makeCDFs(nSMm2_16)
mod2_50, ymod2_50 = makeCDFs(nSMm2_50)
mod2_84, ymod2_84 = makeCDFs(nSMm2_84)
mod3_16, ymod3_16 = makeCDFs(nSMm3_16)
mod3_50, ymod3_50 = makeCDFs(nSMm3_50)
mod3_84, ymod3_84 = makeCDFs(nSMm3_84)
# mod4_16, ymod4_16 = makeCDFs(nSMm4_16)
# mod4_50, ymod4_50 = makeCDFs(nSMm4_50)
# mod4_84, ymod4_84 = makeCDFs(nSMm4_84)


In [ ]:
import seaborn as sns

fig, ax = plt.subplots(1,1, figsize=(6,4))

# ax.plot(mod2, ymod2, label="Hybrid (76% gas + 24% stars)", color='purple', ls="-",lw=2)
sns.ecdfplot(realSNe, color = "dimgrey",  label="Real SNe Sample", ax=ax,lw=2)
sns.ecdfplot(nm1, label="Random Model", color='red', ls=":",ax=ax,lw=2)
sns.ecdfplot(nm2, label="H-alpha", color='green', ls="--",lw=2,ax=ax)
sns.ecdfplot(nm3, label="Stars (3.4 Micron)", color='blue', ls="-.",lw=2,ax=ax)
# ax[0].plot(mod4, ymod4, label="Hybrid (76% gas + 24% stars)", color='purple', ls="-",lw=2)
# ax.plot(mod2, ymod2, label="H-alpha", color='green', ls="--",lw=2)
ax.fill_between(x=mod1_50, y1=ymod1_16,y2=ymod1_84, color='red', alpha=0.2)
ax.fill_between(x=mod2_50, y1=ymod2_16,y2=ymod2_84, color='green', alpha=0.2)
# ax.fill_between(x=mod2_50, y1=ymod2_50,y2=ymod2_84, color='green', alpha=0.2)
ax.fill_between(x=mod3_50, y1=ymod3_16,y2=ymod3_84, color='blue', alpha=0.2)
# ax[0].fill_between(x=mod4_50, y1=ymod4_16,y2=ymod4_84, color='purple', alpha=0.2)
ax.set_xscale('log')
ax.set_ylim (0.3,1.01)
ax.set_xlim(1,2000)
ax.legend(loc='best')

# sns.ecdfplot(nSMIa,color = "black", ls=':', label="Real SNe Ia", ax=ax[1],lw=2)
# sns.ecdfplot(nSMII,color = "black", ls='-',  label="Real SNe II", ax=ax[1],lw=2)
# sns.ecdfplot(nSMIbc,color = "black", ls='--',  label="Real SNe Ibc", ax=ax[1],lw=2)
# sns.ecdfplot(nSMunc,color = "black", ls='-.',  label="Real Unclassified", ax=ax[1],lw=2)

# ax[1].plot(mod1, ymod1, color='red', ls=":",lw=2)
# ax[1].plot(mod2, ymod2, color='green', ls="--",lw=2)
# ax[1].plot(mod3, ymod3, color='blue', ls="-.",lw=2)
# # ax[1].plot(mod4, ymod4, color='purple', ls="-",lw=2)
# ax[1].fill_between(x=mod1_50, y1=ymod1_16,y2=ymod1_84, color='red', alpha=0.2)
# ax[1].fill_between(x=mod2_50, y1=ymod2_16,y2=ymod2_84, color='green', alpha=0.2)
# ax[1].fill_between(x=mod3_50, y1=ymod3_16,y2=ymod3_84, color='blue', alpha=0.2)
# # ax[1].fill_between(x=mod4_50, y1=ymod4_16,y2=ymod4_84, color='purple', alpha=0.2)
# ax[1].set_xscale('log')
# ax[1].set_ylim (0.3,1.01)
# ax[1].set_xlim(60,5000)

# ax[1].legend(loc='best')

ax.tick_params(axis = "both", which = "both", direction = "in")
# ax[1].tick_params(axis = "both", which = "both", direction = "in")

plt.xlabel("Distance [pc]", fontsize = 14)
ax.set_ylabel(r"CDF f $(< d)$", fontsize = 14)
# ax[1].set_ylabel("CDF f (< d)", fontsize = 14)
ax.grid(which='major', alpha=0.5)
# ax[1].grid(which='major', alpha=0.5)
fig.savefig("../Figures/DistanceCDF.png", dpi=300)
plt.show()
plt.close()